In [1]:
import pulp
import pandas as pd
import init_data

In [4]:
# データの読み込み
S, I, V, P, T, t, C, K = init_data.load_data()
# 問題の定義
prob = pulp.LpProblem('karaoke', pulp.LpMaximize)
# 変数の定義
x = pulp.LpVariable.dicts('x', [(i,s,k)for i in I for s in S for k in K], cat='Binary')
# 目的関数の定義
prob += pulp.lpSum([V[(i,s)]*x[(i,s,k)] for i in I for s in S for k in K]) + pulp.lpSum([x[(i,s,k)] for i in I for s in C for k in K])*100
# 制約条件の定義 
# 1. 1つの番目の人は1回に1曲しか歌えない
for k in K:
    prob += pulp.lpSum([x[(i,s,k)] for i in I for s in S]) <= 1
# 2. 順番が若い順になる
for k in K:
    if k != K[-1]:
        prob += pulp.lpSum([x[(i,s,k)] for i in I for s in S]) >= pulp.lpSum([x[(i,s,k+1)] for i in I for s in S])
# 3.時間内に収める
prob += pulp.lpSum([t[s]*x[(i,s,k)] for i in I for s in S for k in K]) <= T
# # 4. 交互に歌う
for i in I:
    for k in K:
        if k != K[-1]:
            prob += pulp.lpSum([x[(i,s,k)] for s in S]) + pulp.lpSum([x[(i,s,k+1)] for s in S])<=1
# #5. 二人で同じ曲は二回歌えない
for s in S:
    prob += pulp.lpSum([x[(i,s,k)] for i in I for k in K]) <= 1
#6. 共通の場合、歌ってる回数が多いほうが歌う
for s in C:
    if P[(1,s)] >= P[(2,s)]:
        prob += pulp.lpSum([x[(1,s,k)] for k in K]) >= pulp.lpSum([x[(2,s,k)] for k in K])
    else:
        prob += pulp.lpSum([x[(2,s,k)] for k in K]) >= pulp.lpSum([x[(1,s,k)] for k in K])
#7. 明はできるだけ連続
#8. 相手がその歌の歌手の歌を歌った回数が多ければ、共通でなくても歌う

In [5]:
#求解
solve =prob.solve()
print(pulp.LpStatus[solve])
sumtime =0
sumc = 0
#結果の表示
for k in K:
    for i in I:
        for s in S:
            if x[(i,s,k)].value() == 1:
                sumtime += t[s]
                if s in C:
                    sumc += 1
                print(f"{i}さんは{k}回目に{s}を歌う")
print(f"合計時間は{sumtime}分です")
print(f"共通の曲は{sumc}曲です")

Optimal
1さんは1回目にフライデーナイトを歌う
2さんは2回目に366日を歌う
1さんは3回目に春よ、来いを歌う
2さんは4回目に君が代を歌う
1さんは5回目に涙そうそうを歌う
2さんは6回目に恋するフォーチュンクッキーを歌う
1さんは7回目にプレタポルテを歌う
2さんは8回目に世界に一つだけの花を歌う
1さんは9回目にシャイニング・オン君が哀しいを歌う
2さんは10回目に恋ダンスを歌う
1さんは11回目に上を向いて歩こうを歌う
2さんは12回目にプライバシーを歌う
1さんは13回目にサイレントマジョリティーを歌う
2さんは14回目にGReeeeNを歌う
1さんは15回目にツバサをくださいを歌う
合計時間は59.50000000000001分です
共通の曲は10曲です


In [10]:
print(sumtime)

59.2
